In [ ]:
# ==========================================
# 1. ADIM: KURULUM VE DRİVE BAĞLANTISI
# ==========================================
from google.colab import drive
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Drive Bağla
drive.mount('/content/drive')

# Gerekli kütüphaneyi kur (MIT-BIH okumak için)
!pip install -q wfdb

import wfdb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ==========================================
# 2. ADIM: VERİ İNDİRME (Senin düzenin)
# ==========================================
# Veriyi senin klasörüne indiriyoruz (zaten varsa indirmez)
data_dir = "/content/drive/MyDrive/mitdb"
os.makedirs(data_dir, exist_ok=True)

# Veri setini kontrol et, yoksa indir
if len(os.listdir(data_dir)) < 10:
    print("MIT-BIH veriseti indiriliyor...")
    !wget -q -r -N -c -np https://physionet.org/files/mitdb/1.0.0/ -P /content/
    !cp -r /content/physionet.org/files/mitdb/1.0.0/* "{data_dir}/"
    print("İndirme tamamlandı.")
else:
    print("MIT-BIH veriseti zaten Drive'da mevcut.")

MIT-BIH veriseti zaten Drive'da mevcut.


In [ ]:
# ==========================================
# 3. ADIM: RR ARALIKLARI + ETİKETLER (MULTI-CLASS + BINARY)
# ==========================================
# Multi-class ön-eğitim için AAMI benzeri etiketleme, ardından binary fine-tune

NORMAL_SYMBOLS = ['N', 'L', 'R', 'e', 'j']
AAMI_MAP = {
    'N': 'N', 'L': 'N', 'R': 'N', 'e': 'N', 'j': 'N',
    'A': 'S', 'a': 'S', 'J': 'S', 'S': 'S',
    'V': 'V', 'E': 'V',
    'F': 'F',
    '/': 'Q', 'f': 'Q', 'Q': 'Q'
}
AAMI_LABELS = {'N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4}
ALLOWED_SYMBOLS = set(AAMI_MAP.keys())


def load_rr_sequences(records, seq_len=10):
    X_all = []
    y_symbols = []
    print("RR aralıkları ve semboller çekiliyor...")
    for record in records:
        try:
            ann = wfdb.rdann(f"{data_dir}/{record}", 'atr')
            rr_intervals = np.diff(ann.sample) / 360.0
            symbols = np.array(ann.symbol[1:])

            min_len = min(len(rr_intervals), len(symbols))
            rr_intervals = rr_intervals[:min_len]
            symbols = symbols[:min_len]

            valid = (rr_intervals > 0.2) & (rr_intervals < 2.0)
            rr_intervals = rr_intervals[valid]
            symbols = symbols[valid]

            for i in range(len(rr_intervals) - seq_len):
                sequence = rr_intervals[i : i + seq_len]
                target_symbol = symbols[i + seq_len]
                if target_symbol not in ALLOWED_SYMBOLS:
                    continue
                X_all.append(sequence)
                y_symbols.append(target_symbol)
        except Exception:
            continue
    return np.array(X_all), np.array(y_symbols)


# Kayıt listesi
all_records = [f.replace('.dat', '') for f in os.listdir(data_dir) if f.endswith('.dat')]
selected_records = all_records  # tamamını kullan

X_seq, y_symbols = load_rr_sequences(selected_records, seq_len=10)
print(f"Toplam sekans: {len(X_seq)}")
unique, counts = np.unique(y_symbols, return_counts=True)
print("Sembol dağılımı:", dict(zip(unique, counts)))


# --- MULTI-CLASS (AAMI) ---
def to_aami_id(sym):
    cls = AAMI_MAP.get(sym)
    if cls is None:
        return None
    return AAMI_LABELS[cls]


X_multi, y_multi = [], []
for seq, sym in zip(X_seq, y_symbols):
    cls_id = to_aami_id(sym)
    if cls_id is not None:
        X_multi.append(seq)
        y_multi.append(cls_id)
X_multi = np.array(X_multi)
y_multi = np.array(y_multi)
print("AAMI sınıf dağılımı:", {k: int(np.sum(y_multi == v)) for k, v in AAMI_LABELS.items()})


# --- BINARY (
# =0, Aritmi=1) ---
y_binary = np.array([0 if s in NORMAL_SYMBOLS else 1 for s in y_symbols])
X_binary = X_seq

arr_mask = y_binary == 1
norm_mask = y_binary == 0
arr_count = int(np.sum(arr_mask))
if arr_count == 0:
    raise ValueError("Aritmi örneği bulunamadı, veri filtresini kontrol et.")

X_norm_down = resample(X_binary[norm_mask], replace=False, n_samples=arr_count, random_state=42)
y_norm_down = np.zeros(arr_count)
X_balanced = np.concatenate([X_norm_down, X_binary[arr_mask]])
y_balanced = np.concatenate([y_norm_down, np.ones(arr_count)])

print(f"Dengelenmiş binary veri: {len(X_balanced)} örnek")


# Eğitim/Test Ayırma
Xmc_train, Xmc_val, ymc_train, ymc_val = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)
Xb_train, Xb_test, yb_train, yb_test = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

# Model giriş formatı: [örnek, zaman, özellik]
Xmc_train = Xmc_train.reshape((-1, 10, 1))
Xmc_val = Xmc_val.reshape((-1, 10, 1))
Xmc_full = X_multi.reshape((-1, 10, 1))
Xb_train = Xb_train.reshape((-1, 10, 1))
Xb_test = Xb_test.reshape((-1, 10, 1))

print("Şekiller ->")
print("Multi-class train:", Xmc_train.shape, "val:", Xmc_val.shape)
print("Binary train:", Xb_train.shape, "test:", Xb_test.shape)

RR aralıkları ve semboller çekiliyor...
Toplam sekans: 108638
Sembol dağılımı: {np.str_('/'): np.int64(6981), np.str_('A'): np.int64(2526), np.str_('E'): np.int64(104), np.str_('F'): np.int64(800), np.str_('J'): np.int64(83), np.str_('L'): np.int64(8038), np.str_('N'): np.int64(74584), np.str_('Q'): np.int64(29), np.str_('R'): np.int64(7092), np.str_('S'): np.int64(2), np.str_('V'): np.int64(7036), np.str_('a'): np.int64(149), np.str_('e'): np.int64(16), np.str_('f'): np.int64(976), np.str_('j'): np.int64(222)}
AAMI sınıf dağılımı: {'N': 89952, 'S': 2760, 'V': 7140, 'F': 800, 'Q': 7986}
Dengelenmiş binary veri: 37372 örnek
Şekiller ->
Multi-class train: (86910, 10, 1) val: (21728, 10, 1)
Binary train: (29897, 10, 1) test: (7475, 10, 1)


In [ ]:
# ==========================================
# 4. ADIM: CNN BACKBONE ÖN-EĞİTİMİ (MULTI-CLASS)
# ==========================================

def create_cnn_backbone(input_shape=(10, 1)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(32, 3, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    return Model(inputs, x, name="rr_cnn_backbone")


def build_multiclass_model(backbone, num_classes):
    x = backbone.output
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return Model(backbone.input, outputs, name="rr_cnn_multiclass")


backbone = create_cnn_backbone(input_shape=(10, 1))
mc_model = build_multiclass_model(backbone, num_classes=len(AAMI_LABELS))

mc_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("\nCNN backbone multi-class ön-eğitimi başlıyor...")
history_mc = mc_model.fit(
    Xmc_train,
    ymc_train,
    validation_data=(Xmc_val, ymc_val),
    epochs=12,
    batch_size=64,
    verbose=1,
)

backbone_weights_path = '/content/drive/MyDrive/mitdb/cnn_rr_backbone.weights.h5'
backbone.save_weights(backbone_weights_path)
print(f"Backbone ağırlıkları kaydedildi: {backbone_weights_path}")


CNN backbone multi-class ön-eğitimi başlıyor...
Epoch 1/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.8208 - loss: 0.5369 - val_accuracy: 0.8425 - val_loss: 0.4277
Epoch 2/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8447 - loss: 0.4308 - val_accuracy: 0.8610 - val_loss: 0.3883
Epoch 3/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8554 - loss: 0.4080 - val_accuracy: 0.8499 - val_loss: 0.5111
Epoch 4/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8621 - loss: 0.3900 - val_accuracy: 0.8508 - val_loss: 0.3903
Epoch 5/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8667 - loss: 0.3818 - val_accuracy: 0.8502 - val_loss: 0.4088
Epoch 6/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8709 - loss: 0.3747 - val_accuracy: 0.8472 - val_loss: 0.5184
Epoch 7/12
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8752 - loss: 0.3663 - val_accuracy: 0.8635 - val_loss: 0.3713
Epoch 8/12
1358/1358 ━━━━━━━━━━━━━━━━━━

In [ ]:
# ==========================================
# 5. ADIM: TRANSFER LEARNING (BINARY FINE-TUNE)
# ==========================================
# Backbone'u yeniden kur, ağırlıkları yükle ve donmuş halde binary kafa eğit.

backbone_ft = create_cnn_backbone(input_shape=(10, 1))
backbone_ft.load_weights(backbone_weights_path)

for layer in backbone_ft.layers:
    layer.trainable = False

x = backbone_ft.output
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
binary_output = layers.Dense(1, activation='sigmoid')(x)
transfer_model = Model(backbone_ft.input, binary_output, name="rr_cnn_transfer")

transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

print("\nTransfer learning (backbone donmuş) başlıyor...")
history_frozen = transfer_model.fit(
    Xb_train,
    yb_train,
    validation_data=(Xb_test, yb_test),
    epochs=8,
    batch_size=64,
    verbose=1,
)

# Üst katmanları açıp daha düşük LR ile fine-tune
for layer in backbone_ft.layers[-2:]:
    layer.trainable = True

transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

print("\nİnce ayar (üst conv katmanları açık)...")
history_unfrozen = transfer_model.fit(
    Xb_train,
    yb_train,
    validation_data=(Xb_test, yb_test),
    epochs=6,
    batch_size=64,
    verbose=1,
)


Transfer learning (backbone donmuş) başlıyor...
Epoch 1/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7203 - loss: 0.5486 - val_accuracy: 0.7813 - val_loss: 0.4616
Epoch 2/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7846 - loss: 0.4608 - val_accuracy: 0.7946 - val_loss: 0.4396
Epoch 3/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7968 - loss: 0.4359 - val_accuracy: 0.7969 - val_loss: 0.4308
Epoch 4/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8029 - loss: 0.4309 - val_accuracy: 0.7980 - val_loss: 0.4237
Epoch 5/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8067 - loss: 0.4223 - val_accuracy: 0.8047 - val_loss: 0.4197
Epoch 6/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8113 - loss: 0.4149 - val_accuracy: 0.8103 - val_loss: 0.4149
Epoch 7/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8151 - loss: 0.4120 - val_accuracy: 0.8123 - val_loss: 0.4120
Epoch 8/8
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy:

In [ ]:
# ==========================================
# 6. ADIM: DEĞERLENDİRME
# ==========================================
from sklearn.metrics import classification_report, confusion_matrix

y_pred_prob = transfer_model.predict(Xb_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nModel Performans Raporu (Transfer):")
print(classification_report(yb_test, y_pred, target_names=['Normal', 'Aritmi']))

cm = confusion_matrix(yb_test, y_pred)
print("Confusion Matrix:\n", cm)

234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Model Performans Raporu (Transfer):
              precision    recall  f1-score   support

      Normal       0.81      0.83      0.82      3738
      Aritmi       0.82      0.80      0.81      3737

    accuracy                           0.81      7475
   macro avg       0.81      0.81      0.81      7475
weighted avg       0.81      0.81      0.81      7475

Confusion Matrix:
 [[3086  652]
 [ 745 2992]]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Confusion Matrix'i görselleştirme
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Normal', 'Aritmi'], yticklabels=['Normal', 'Aritmi'])
plt.title('Confusion Matrix Heatmap')
plt.xlabel('Tahmin Edilen Etiket')
plt.ylabel('Gerçek Etiket')
plt.show()

In [ ]:
# ==========================================
# 7. ADIM: MODELİ KAYDETME
# ==========================================
final_path = '/content/drive/MyDrive/mitdb/cnn_rr_arrhythmia_transfer.h5'
transfer_model.save(final_path)
print(f"\n✅ Transfer modeli kaydedildi: {final_path}")
print(f"Backbone ağırlıkları: {backbone_weights_path}")


✅ Transfer modeli kaydedildi: /content/drive/MyDrive/mitdb/cnn_rr_arrhythmia_transfer.h5
Backbone ağırlıkları: /content/drive/MyDrive/mitdb/cnn_rr_backbone.weights.h5
